# 🪟 Sliding Window Technique Mastery

**Phase 3: Interview Preparation - Coding Problems Module 2**

**Prerequisites**: Array manipulation, basic pattern recognition

**Master the essential sliding window patterns that solve 40% of LeetCode problems!**

---

## 🏗️ Sliding Window Fundamentals

Thefour sliding window patterns with implementation templates

In [ ]:
// Sliding Window Pattern Framework
sealed trait WindowType
case object FixedSize extends WindowType
case object VariableSize extends WindowType
case object MaximumOptimization extends WindowType
case object MinimumOptimization extends WindowType

case class WindowResult[T](
  value: T,
  windowStart: Int,
  windowEnd: Int,
  windowSize: Int,
  operations: Int,
  timeComplexity: String,
  spaceComplexity: String
)

// Template for fixed-size sliding window
def fixedWindowTemplate[T](
  data: Array[T],
  windowSize: Int,
  combineFn: Array[T] => T
): WindowResult[T] = {
  var operations = 0
  var maxResult = combineFn(data.take(windowSize))
  var bestStart = 0
  
  operations += windowSize
  
  // Maintain a window of size k
  for (i <- windowSize until data.length) {
    operations += 1
    // Remove element from beginning of window
    // Add new element to end of window
    val currentResult = combineFn(data.slice(i - windowSize + 1, i + 1))
    
    // Update maximum if needed
    val comparison = data(0) // This would be the comparison logic
    if (currentResult == maxResult) { // Simplified for template
      maxResult = currentResult
      bestStart = i - windowSize + 1
    }
  }
  
  WindowResult(
    maxResult,
    bestStart,
    bestStart + windowSize - 1,
    windowSize,
    operations,
    "O(n×k)",
    "O(1)"
  )
}

// Enhanced sliding window implementation tracker
class SlidingWindowTracker {
  private var totalOperations = 0
  private var windowAdjustments = 0
  private var evaluations = 0
  
  def recordOperation(op: String): Unit = {
    totalOperations += 1
    println(s"Operation: $op (Total: $totalOperations)")
  }
  
  def recordWindowAdjustment(reason: String): Unit = {
    windowAdjustments += 1
    println(s"Window adjustment #$windowAdjustments: $reason")
  }
  
  def recordEvaluation(state: String): Unit = {
    evaluations += 1
    println(s"Evaluation #$evaluations: $state")
  }
  
  def getStats(): (Int, Int, Int) = (totalOperations, windowAdjustments, evaluations)
}

println("Sliding Window Framework Initialized")
println("Supports: Fixed-size, Variable-size, Maximum/Minimum optimizations")
println()

## 🏹 LeetCode 209: Minimum Size Subarray Sum (Variable Window)

Find minimum subarray length with sum ≥ k

In [ ]:
// Minimum Size Subarray Sum - Variable window size
def minSubarrayLen(target: Int, nums: Array[Int]): WindowResult[Int] = {
  val tracker = new SlidingWindowTracker
  val n = nums.length
  
  tracker.recordOperation("Initialize window variables")
  var minLength = Int.MaxValue
  var currentSum = 0
  var left = 0
  var bestStart = -1
  var bestEnd = -1
  
  // Expand right pointer
  for (right <- 0 until n) {
    tracker.recordOperation(s"Expand window: add nums[$right] = ${nums(right)}")
    currentSum += nums(right)
    
    tracker.recordEvaluation(s"Current sum: $currentSum, target: $target")
    
    // Contract left pointer while sum >= target
    while (currentSum >= target && left <= right) {
      tracker.recordOperation(s"Contract window: window [$left, $right]")
      
      val currentLength = right - left + 1
      if (currentLength < minLength) {
        minLength = currentLength
        bestStart = left
        bestEnd = right
        tracker.recordOperation(s"New minimum length: $minLength")
      }
      
      tracker.recordOperation(s"Remove nums[$left] = ${nums(left)} from window")
      currentSum -= nums(left)
      left += 1
      
      tracker.recordWindowAdjustment("Contracted left pointer, sum reduced")
    }
  }
  
  val resultLength = if (minLength == Int.MaxValue) 0 else minLength
  
  val (operations, adjustments, evaluations) = tracker.getStats
  tracker.recordOperation(s"Algorithm completed with $operations operations")
  
  WindowResult(
    resultLength,
    bestStart,
    bestEnd,
    if (resultLength > 0) resultLength else 0,
    operations,
    "O(n)",
    "O(1)"
  )
}

// Test cases with detailed tracing
val minSubarrayTests = List(
  (7, Array(2, 3, 1, 2, 4, 3)),  // Expected: 2 ([4,3])
  (4, Array(1, 4, 4)),           // Expected: 1 ([4])
  (11, Array(1, 1, 1, 1, 1, 1, 1, 1)), // Expected: 0
  (3, Array(1, 2, 3))            // Expected: 1 ([3])
)

println("=== Minimum Size Subarray Sum ===")
println("Variable window: Expand right, contract left when condition met\n")

minSubarrayTests.zipWithIndex.foreach { case ((target, nums), idx) =>
  println(s"Test ${idx+1}: Target=$target, Array=${nums.mkString("[", ", ", "]")}")
  
  val result = minSubarrayLen(target, nums)
  val windowElements = if (result.windowStart >= 0) 
    nums.slice(result.windowStart, result.windowEnd + 1).mkString("[", ", ", "]")
  else "No solution"
  
  println(s"Result: Length=${result.value}, Window elements: $windowElements")
  println(s"Performance: ${result.operations} operations, ${result.timeComplexity} time")
  
  // Verification
  if (result.value > 0) {
    val sum = nums.slice(result.windowStart, result.windowEnd + 1).sum
    println(f"Verification: Sum of window = $sum%d (>= target $target%d? ${sum >= target}%b)")
  }
  println("─" * 60)
}
println()

## 🏃 LeetCode 3: Longest Substring Without Repeating Characters

Find the longest substring without duplicate characters

In [ ]:
// Longest Substring Without Repeating Characters
def lengthOfLongestSubstring(s: String): WindowResult[(Int, String)] = {
  val tracker = new SlidingWindowTracker
  val n = s.length
  
  tracker.recordOperation("Initialize sliding window variables")
  var maxLength = 0
  var longestSubstring = ""
  var left = 0
  val charSet = collection.mutable.Set[Char]()
  
  for (right <- 0 until n) {
    val currentChar = s(right)
    
    tracker.recordOperation(s"Expand window right: s[$right] = '$currentChar'")
    
    // Contract window from left until we remove duplicate
    while (charSet.contains(currentChar) && left <= right) {
      val leftChar = s(left)
      tracker.recordOperation(s"Contract window left: remove '$leftChar' at position $left")
      charSet.remove(leftChar)
      left += 1
      tracker.recordWindowAdjustment("Removed duplicate character")
    }
    
    // Add current character
    charSet.add(currentChar)
    tracker.recordOperation(s"Added '$currentChar' to character set")
    
    // Update maximum if needed
    val currentLength = right - left + 1
    val currentSubstring = s.substring(left, right + 1)
    
    tracker.recordEvaluation(s"Current window [$left,$right]: '$currentSubstring' (length $currentLength)")
    
    if (currentLength > maxLength) {
      maxLength = currentLength
      longestSubstring = currentSubstring
      tracker.recordOperation(s"New maximum length: $maxLength")
    }
  }
  
  val (operations, adjustments, evaluations) = tracker.getStats
  
  WindowResult(
    (maxLength, longestSubstring),
    0, 0, maxLength,
    operations,
    "O(n)",
    "O(min(n, unique_chars))"
  )
}

// Alternative implementation with ASCII array
def lengthOfLongestSubstringASCII(s: String): WindowResult[(Int, String)] = {
  val n = s.length
  val lastSeen = Array.fill(256)(-1)
  var maxLength = 0
  var longestSubstring = ""
  var left = 0
  
  for (right <- 0 until n) {
    val currentChar = s(right)
    val lastIndex = lastSeen(currentChar)
    
    // If character seen before and within current window
    if (lastIndex >= left) {
      left = lastIndex + 1
    }
    
    // Update last seen index
    lastSeen(currentChar) = right
    
    // Check current window
    val currentLength = right - left + 1
    if (currentLength > maxLength) {
      maxLength = currentLength
      longestSubstring = s.substring(left, right + 1)
    }
  }
  
  WindowResult(
    (maxLength, longestSubstring),
    0, 0, maxLength,
    n,
    "O(n)",
    "O(256) = O(1)"
  )
}

// Test cases
val substringTests = List(
  "abcabcbb",    // Expected: 3 ("abc")
  "bbbbb",       // Expected: 1 ("b")
  "pwwkew",      // Expected: 3 ("wke")
  "",           // Expected: 0 ("")
  "au",         // Expected: 2 ("au")
  "dvdf",       // Expected: 3 ("vdf")
  "abba"        // Expected: 2 ("ab" or "ba")
)

println("=== Longest Substring Without Repeating Characters ===")
println("Sliding window with Set: Contract when duplicate found\n")

substringTests.foreach { s =>
  println(s"String: \"$s\"")
  
  // Test both implementations
  val (length, substring) = lengthOfLongestSubstring(s).value
  val (asciiLength, asciiSubstring) = lengthOfLongestSubstringASCII(s).value
  
  println(s"Result: Length=$length, Substring=\"$substring\"")
  println(s"ASCII:  Length=$asciiLength, Substring=\"$asciiSubstring\"")
  
  // Verification
  val hasDuplicates1 = substring.toSet.size < substring.length
  val hasDuplicates2 = asciiSubstring.toSet.size < asciiSubstring.length
  println(s"Verification: No duplicates in result = ${!hasDuplicates1 && !hasDuplicates2}")
  
  println("─" * 50)
}
println()

## 🌟 LeetCode 76: Minimum Window Substring (Hard - Fixed Window Concept)

Find minimum window containing all characters of another string

In [ ]:
// Minimum Window Substring - Contains all characters from pattern
def minWindow(s: String, t: String): WindowResult[String] = {
  val tracker = new SlidingWindowTracker
  
  tracker.recordOperation(s"Find minimum window of '$s' containing all chars from '$t'")
  
  if (t.isEmpty) {
    return WindowResult("", 0, -1, 0, 1, "O(1)", "O(1)")
  }
  
  // Count characters needed from t
  val tCount = collection.mutable.Map[Char, Int]()
  val windowCount = collection.mutable.Map[Char, Int]()
  
  for (char <- t) {
    tCount(char) = tCount.getOrElse(char, 0) + 1
  }
  
  val required = tCount.size
  var have = 0
  var left = 0
  var minLength = Int.MaxValue
  var minStart = 0
  
  tracker.recordEvaluation(s"Need $required unique characters from pattern")
  
  // Expand window with right pointer
  for (right <- s.indices) {
    val rightChar = s(right)
    tracker.recordOperation(s"Expand right: s[$right] = '$rightChar'")
    
    windowCount(rightChar) = windowCount.getOrElse(rightChar, 0) + 1
    
    // Check if this character satisfies a requirement
    if (tCount.contains(rightChar) && windowCount(rightChar) == tCount(rightChar)) {
      have += 1
      tracker.recordOperation(s"Satisfied requirement for '$rightChar'")
    }
    
    // Contract window from left while all requirements satisfied
    while (have == required && left <= right) {
      tracker.recordOperation(s"Window [$left,$right] contains all characters")
      
      val currentLength = right - left + 1
      if (currentLength < minLength) {
        minLength = currentLength
        minStart = left
        tracker.recordOperation(s"New minimum window length: $minLength")
      }
      
      // Remove leftmost character
      val leftChar = s(left)
      tracker.recordOperation(s"Contract left: removing '$leftChar' at position $left")
      
      // Update window count
      windowCount(leftChar) = windowCount(leftChar) - 1
      
      // Check if we lost a requirement
      if (tCount.contains(leftChar) && windowCount(leftChar) < tCount(leftChar)) {
        have -= 1
        tracker.recordWindowAdjustment("Lost requirement, stopping contraction")
      }
      
      left += 1
    }
  }
  
  val result = if (minLength == Int.MaxValue) "" else s.substring(minStart, minStart + minLength)
  
  val (operations, adjustments, evaluations) = tracker.getStats
  
  WindowResult(
    result,
    minStart,
    if (result.nonEmpty) minStart + minLength - 1 else -1,
    minLength,
    operations,
    "O(n)",
    "O(unique_chars)"
  )
}

// Test minimum window substring
val windowTests = List(
  ("ADOBECODEBANC", "ABC"),     // Expected: "BANC"
  ("a", "a"),                   // Expected: "a"
  ("a", "aa"),                  // Expected: ""
  ("aa", "aa"),                 // Expected: "aa"
  ("bdab", "ab"),               // Expected: "ab"
  ("aaaaaaaaaaaabbbbbcdd", "abcdd")  // Expected: "abbbbbcdd"
)

println("=== Minimum Window Substring ===")
println("Sliding window: Find smallest window containing all pattern characters\n")

windowTests.foreach { case (s, t) =>
  println(s"Source: \"$s\"\nPattern: \"$t\"")
  
  val result = minWindow(s, t)
  val containsAllChars = if (result.value.nonEmpty) {
    val sourceFreq = result.value.toSeq.groupBy(identity).mapValues(_.length)
    val patternFreq = t.toSeq.groupBy(identity).mapValues(_.length)
    patternFreq.forall { case (char, count) => 
      sourceFreq.getOrElse(char, 0) >= count
    }
  } else t.isEmpty
  
  println(s"Minimum window: \"${result.value}\" (length ${result.value.length})")
  println(s"Window position: [${result.windowStart}, ${result.windowEnd}]")
  println(s"Verification: Contains all pattern chars = $containsAllChars")
  
  if (result.value.nonEmpty && !containsAllChars) {
    println("❌ ERROR: Window does not contain all required characters")
  }
  
  println("─" * 70)
}
println()

## 📈 LeetCode 239: Sliding Window Maximum (Deque-Based)

Find maximum in each sliding window of size k

In [ ]:
// Sliding Window Maximum using Deque
def maxSlidingWindow(nums: Array[Int], k: Int): Array[Int] = {
  if (nums.isEmpty || k <= 0 || k > nums.length) return Array.emptyIntArray
  
  val result = collection.mutable.ArrayBuffer[Int]()
  val deque = collection.mutable.ArrayDeque[Int]() // Store indices
  
  for (i <- nums.indices) {
    // Remove elements outside current window
    while (deque.nonEmpty && deque.head < i - k + 1) {
      deque.removeHead()
    }
    
    // Remove smaller elements from back
    while (deque.nonEmpty && nums(deque.last) <= nums(i)) {
      deque.removeLast()
    }
    
    // Add current element index
    deque.append(i)
    
    // Add maximum to result when window is complete
    if (i >= k - 1) {
      result.append(nums(deque.head))
    }
  }
  
  result.toArray
}

// Naive implementation for comparison (O(n*k))
def maxSlidingWindowNaive(nums: Array[Int], k: Int): Array[Int] = {
  if (nums.isEmpty || k <= 0 || k > nums.length) return Array.emptyIntArray
  
  (for (i <- 0 to nums.length - k) yield {
    nums.slice(i, i + k).max
  }).toArray
}

// Test sliding window maximum
val slidingMaxTests = List(
  (Array(1, 3, -1, -3, 5, 3, 6, 7), 3),     // Expected: [3,3,5,5,6,7]
  (Array(1), 1),                               // Expected: [1]
  (Array(1, -1), 1),                           // Expected: [1,-1]
  (Array(9, 11), 2),                           // Expected: [11]
  (Array(4, -2), 2)                            // Expected: [4]
)

println("=== Sliding Window Maximum ===")
println("Using deque to maintain indices in decreasing order\n")

slidingMaxTests.foreach { case (nums, k) =>
  println(s"Array: ${nums.mkString("[", ", ", "]")}, Window size: $k")
  
  val optimized = maxSlidingWindow(nums, k)
  val naive = maxSlidingWindowNaive(nums, k)
  
  println(s"Optimized (O(n)): ${optimized.mkString("[", ", ", "]")}")
  println(s"Naive (O(n*k)):   ${naive.mkString("[", ", ", "]")}")
  
  val resultsMatch = optimized.sameElements(naive)
  println(s"Results match: $resultsMatch\n")
  
  if (!resultsMatch) {
    println("❌ ERROR: Results do not match!")
  }
  
  println("─" * 50)
}
println()

## 🏹 LeetCode 424: Longest Repeating Character Replacement

Find longest substring after replacing k characters

In [ ]:
// Longest Repeating Character Replacement
def characterReplacement(s: String, k: Int): WindowResult[(Int, String)] = {
  val tracker = new SlidingWindowTracker
  
  tracker.recordOperation(s"Replace up to $k characters to maximize repeated chars in '$s'")
  
  val n = s.length
  if (n <= 1) {
    return WindowResult((Math.min(n, k + 1), s), 0, n-1, n, 1, "O(1)", "O(1)")
  }
  
  val freqCount = collection.mutable.Map[Char, Int]()
  var maxLength = 0
  var longestSubstring = ""
  var left = 0
  var maxFreq = 0
  
  for (right <- 0 until n) {
    val rightChar = s(right)
    freqCount(rightChar) = freqCount.getOrElse(rightChar, 0) + 1
    maxFreq = math.max(maxFreq, freqCount(rightChar))
    
    tracker.recordOperation(s"Expand right: s[$right] = '$rightChar', maxFreq=$maxFreq")
    
    val currentLength = right - left + 1
    val changesNeeded = currentLength - maxFreq
    
    tracker.recordEvaluation(s"Window [$left,$right]: length=$currentLength, maxFreq=$maxFreq, changesNeeded=$changesNeeded")
    
    if (changesNeeded <= k) {
      // Valid window
      if (currentLength > maxLength) {
        maxLength = currentLength
        longestSubstring = s.substring(left, right + 1)
        tracker.recordOperation(s"New maximum length: $maxLength")
      }
    } else {
      // Contract from left
      val leftChar = s(left)
      tracker.recordOperation(s"Contract left: remove '$leftChar' at position $left")
      
      freqCount(leftChar) = freqCount(leftChar) - 1
      left += 1
      tracker.recordWindowAdjustment("Reduced window size to allow replacements")
    }
  }
  
  val (operations, adjustments, evaluations) = tracker.getStats
  
  WindowResult(
    (maxLength, longestSubstring),
    0, 0, maxLength,
    operations,
    "O(n)",
    "O(unique_chars)"
  )
}

// Test character replacement
val replacementTests = List(
  ("ABAB", 2),      // Expected: 4
  ("AABABBA", 1),   // Expected: 4
  ("AAAA", 2),      // Expected: 4
  ("ABCDE", 1),     // Expected: 2
  ("AABABBA", 2)    // Expected: 5
)

println("=== Longest Repeating Character Replacement ===")
println("Sliding window: Maximize repeated characters with limited replacements\n")

replacementTests.foreach { case (s, k) =>
  println(s"String: \"$s\", Max replacements: $k")
  
  val result = characterReplacement(s, k)
  val (length, substring) = result.value
  
  println(s"Result: Length=$length, Substring=\"$substring\"")
  
  // Verify: count replacements needed
  val charCounts = substring.groupBy(identity).mapValues(_.length)
  val mostFrequent = charCounts.values.max
  val replacementsNeeded = substring.length - mostFrequent
  
  println(f"Verification: $replacementsNeeded%d replacements needed (≤ k=$k%d: ${replacementsNeeded <= k}%b)")
  
  if (replacementsNeeded > k) {
    println("❌ ERROR: Too many replacements needed!")
  }
  
  println("─" * 50)
}
println()

## 📊 Sliding Window Pattern Analysis

### **Four Major Patterns:**
1. **Fixed Window Size**: Sum, average, maximum of fixed windows
2. **Variable Window Size**: Minimum/maximum window with conditions
3. **Expanding Only**: Grow window until condition, then start over
4. **Two Pointers**: Similar to variable window but simplified

### **Time Complexity:**
- **Best**: O(n) for all problems above
- **Worst**: O(n) with constant-time operations per element
- **Space**: O(1) to O(n) depending on frequency tracking

### **Common Optimization Techniques:**
- **Deque**: Maintain indices for efficient maximum/minimum
- **Frequency Maps**: Track character/element counts
- **HashSet**: Track unique elements (bounded problems)
- **Prefix Sums**: Precompute sums for O(1) range queries

### **Interview Distinctions:**
- "Why is sliding window efficient vs nested loops?"
- "When should I use a variable vs fixed window?"
- "How does the O(1) deque optimization work?"
- "What's the relationship to two pointers technique?"

### **Advanced Applications:**
- **Rate Limiting**: Sliding window counters
- **Traffic Shaping**: Buffer management algorithms
- **Log Analysis**: Time-window aggregations
- **Real-time Processing**: Stream processing windows

**Next: Fast & Slow pointers for cycle detection and linked list problems**